# GEMINI RETURNING MODELS

In [8]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("C:\\Users\\MUSTAFA\\Desktop\\Chatbot\\data\\iade.pdf")
data = loader.load()
print(data)

[Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-12-30T18:39:08+01:00', 'moddate': '2025-12-30T18:39:08+01:00', 'source': 'C:\\Users\\MUSTAFA\\Desktop\\Chatbot\\data\\iade.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='KİTAP DÜNYASI - KAPSAMLI İADE, DEĞİŞİM VE CAYMA POLİTİKASI PROSEDÜRÜ \n(V.2025.2) \n \n1. İADE SÜRESİ Müşterilerimiz, satın aldıkları ürünleri teslim aldıkları \ntarihten itibaren 14 gün içinde sebep göstermeksizin iade etme hakkına \nsahiptir. 14 günü geçen talepler sistem tarafından otomatik olarak \nreddedilir ve işleme alınmaz. \n \n2. İADE KOŞULLARI \n \nÜrünün ambalajı açılmamış, bozulmamış ve kullanılmamış olmalıdır. \n \nJelatini/paketi açılmış kitap, CD, DVD, Blu-Ray gibi kopyalanabilir \nürünlerin iadesi, Fikir ve Sanat Eserleri Kanunu gereği kesinlikle kabul \nedilmemektedir. \n \nÜrünle birlikte orijinal fatura (veya e-fatura çıktısı) gönderilmelidir. \nFaturasız gönderilen 

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500)
docs = text_splitter.split_documents(data)
load_dotenv()

True

In [14]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
vector = embeddings.embed_query("hello, world!")
vector[:5]

[0.014134909, -0.022324152, -0.05460342, -0.0062845494, -0.033924025]

In [15]:
from langchain_chroma import Chroma
docsearch = Chroma.from_documents(data, embeddings)
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":2})

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    max_tokens=500
)

In [17]:
system_prompt = (
    "You are assistant for question-answering tasks"
    "Use the following pieces of retrieved context to answer"
    "If you dont't know the answer, say that you don't know"
    "Use three sentences maximum and keep the answer concise"
    "\n\n"
    "{context}"
)

In [18]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [19]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain


question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)
